# Apple 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt


In [2]:
train_dir = 'train'
validation_dir = 'validation'


In [3]:
train_scab_dir = os.path.join(train_dir, 'Apple___Apple_scab')  
train_black_dir = os.path.join(train_dir, 'Apple___Black_rot')  
train_rust_dir = os.path.join(train_dir, 'Apple___Cedar_apple_rust')  
train_healthy_dir = os.path.join(train_dir, 'Apple___healthy')  

validation_scab_dir = os.path.join(validation_dir, 'Apple___Apple_scab')  
validation_black_dir = os.path.join(validation_dir, 'Apple___Black_rot')  
validation_rust_dir = os.path.join(validation_dir, 'Apple___Cedar_apple_rust')  
validation_healthy_dir = os.path.join(validation_dir, 'Apple___healthy')  



In [4]:
num_scab_tr = len(os.listdir(train_scab_dir))
num_black_tr = len(os.listdir(train_black_dir))
num_rust_tr = len(os.listdir(train_rust_dir))
num_healthy_tr = len(os.listdir(train_healthy_dir))

num_scab_val = len(os.listdir(validation_scab_dir))
num_black_val = len(os.listdir(validation_black_dir))
num_rust_val = len(os.listdir(validation_rust_dir))
num_healthy_val = len(os.listdir(validation_healthy_dir))


total_train = num_scab_tr + num_black_tr+num_rust_tr+num_healthy_tr
total_val = num_scab_val + num_black_val+num_rust_val+num_healthy_val


In [5]:
print("Total training images:", total_train)
print("Total validation images:", total_val)

Total training images: 10149
Total validation images: 2537


In [6]:
batch_size = 128
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224


In [7]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data


In [8]:
training_set = train_image_generator.flow_from_directory(train_dir,
                                                 target_size=(224, 224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

test_set = validation_image_generator.flow_from_directory(validation_dir,
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical')


Found 10148 images belonging to 4 classes.
Found 2536 images belonging to 4 classes.


In [9]:
# Building CNN based on AlexNet Architecture

# Importing Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

# Initializing the CNN
classifier = Sequential()

# Convolution Step 1
classifier.add(Convolution2D(96, 11, strides = (4, 4), padding = 'valid', input_shape=(224, 224, 3), activation = 'relu'))

# Max Pooling Step 1
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Convolution Step 2
classifier.add(Convolution2D(256, 11, strides = (1, 1), padding='valid', activation = 'relu'))

# Max Pooling Step 2
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding='valid'))
classifier.add(BatchNormalization())

# Convolution Step 3
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 4
classifier.add(Convolution2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'))
classifier.add(BatchNormalization())

# Convolution Step 5
classifier.add(Convolution2D(256, 3, strides=(1,1), padding='valid', activation = 'relu'))

# Max Pooling Step 3
classifier.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'valid'))
classifier.add(BatchNormalization())

# Flattening Step
classifier.add(Flatten())

# Full Connection Step
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(Dense(units = 4, activation = 'softmax'))
classifier.summary()


# Compiling the CNN
classifier.compile(optimizer=optimizers.SGD(lr=0.001, momentum=0.9, decay=0.005),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 384)        

In [ ]:
history = classifier.fit_generator(training_set,
                                   steps_per_epoch=training_set.samples//batch_size,
                                   validation_data=test_set,
                                   epochs=epochs,
                                   validation_steps=test_set.samples//batch_size)

Epoch 1/50
79/79 [==============================] - 567s 7s/step - loss: 1.0757 - accuracy: 0.6063 - val_loss: 1.3006 - val_accuracy: 0.3775
Epoch 2/50
79/79 [==============================] - 569s 7s/step - loss: 0.5851 - accuracy: 0.7958 - val_loss: 1.0752 - val_accuracy: 0.5411
Epoch 3/50
79/79 [==============================] - 568s 7s/step - loss: 0.4223 - accuracy: 0.8479 - val_loss: 0.9104 - val_accuracy: 0.5689
Epoch 4/50
79/79 [==============================] - 562s 7s/step - loss: 0.3475 - accuracy: 0.8744 - val_loss: 0.6546 - val_accuracy: 0.7716
Epoch 5/50
79/79 [==============================] - 576s 7s/step - loss: 0.2792 - accuracy: 0.9006 - val_loss: 0.5697 - val_accuracy: 0.8343
Epoch 6/50
79/79 [==============================] - 572s 7s/step - loss: 0.2530 - accuracy: 0.9085 - val_loss: 0.6101 - val_accuracy: 0.7542
Epoch 7/50
79/79 [==============================] - 562s 7s/step - loss: 0.2175 - accuracy: 0.9242 - val_loss: 0.4198 - val_accuracy: 0.8862
Epoch 8/50
79

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
